In [ ]:
!pip install pdfplumber

     |████████████████████████████████| 5.6MB 6.2MB/s 
     |████████████████████████████████| 143kB 34.9MB/s 
     |████████████████████████████████| 13.7MB 244kB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.23-cp36-none-any.whl size=29176 sha256=19a7f314ea166d6d2ddacc657d093f76c5847fcdc16aeee09c62224b4cdbea84
  Stored in directory: /root/.cache/pip/wheels/f6/4d/ae/abb7b92fbed5d09a835c259f26d2a3223ea929a0ad6d322097
Successfully built pdfplumber


In [ ]:
import re

import requests
import pdfplumber

import pandas as pd
from collections import namedtuple

In [ ]:
Inv = namedtuple('Inv', 'vend_num vend_name inv_dt due_dt inv_amt net_amt description')


VEND_NUM = 19

In [ ]:
def download_file(url):
  local_filename = url.split('/')[-1]

  with requests.get(url) as r:
    with open(local_filename, 'wb') as f:
      f.write(r.content)

  return local_filename

In [ ]:
iv_url = 'https://www.tabs3.com/support/sample/apreports.pdf'

In [ ]:
iv = download_file(iv_url)
print(iv)

apreports.pdf


In [ ]:
with pdfplumber.open(iv) as pdf:
    page = pdf.pages[15]
    text = page.extract_text()

In [ ]:
print(text)

AccountsPayableReportPack
changeswillbereflectedunderthe"New"heading.Whenaninvoiceormanualcheckisdeleted,
theInvoiceAmountisautomaticallychangedtozeroandtheStatusfieldischangedto"D".
TotalTransactionCount Totalnumberoftransactionsonthelistincludingchangesanddeletions.Eachchangedtransaction
(i.e.,botholdandnew)countsasonetransaction.
1Notshownonsamplereport.
Invoice by Vendor List
Date: 08/25/2020 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenanc

In [ ]:
new_vend_re = re.compile(r'^\d{3} [A-Z].*')

In [ ]:
for line in text.split('\n'):
    if new_vend_re.match(line):
     print(line)

200 United Parcel Service
202 Software Technology, LLC
203 Clerk of the County Court
204 Clerk of the District Court
225 Jackson/Wylinda
240 NELCO
250 D & B Real Estate Management Company
300 Thomson Reuters Payment Center
325 Professional Messenger Services
400 Sprint Local & Long Distance
700 Clean All Janitorial Services


In [ ]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)
        print(vend_num)
        print(vend_name)

200
United Parcel Service
202
Software Technology, LLC
203
Clerk of the County Court
204
Clerk of the District Court
225
Jackson/Wylinda
240
NELCO
250
D & B Real Estate Management Company
300
Thomson Reuters Payment Center
325
Professional Messenger Services
400
Sprint Local & Long Distance
700
Clean All Janitorial Services


In [ ]:
inv_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]')

In [ ]:
line_items = []
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)    
    
    line = inv_line_re.search(line)
    if line:
        inv_dt = line.group(1)
        due_dt = line.group(2)
        inv_amt = line.group(3)
        net_amt = line.group(4)
        desc = line.group(5)
        line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))

In [ ]:
df = pd.DataFrame(line_items)
df.head()

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,032620,062620,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032720,062720,75.00,75.00,Postage
2,200,United Parcel Service,072620,072620,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",082020,082520,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [ ]:
df.head()

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,032620,062620,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032720,062720,75.00,75.00,Postage
2,200,United Parcel Service,072620,072620,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",082020,082520,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [ ]:
df['inv_dt'] = pd.to_datetime(df['inv_dt'])
df['due_dt'] = pd.to_datetime(df['due_dt'])

In [ ]:
df.head()

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,2020-03-26,2020-06-26,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,2020-03-27,2020-06-27,75.00,75.00,Postage
2,200,United Parcel Service,2020-07-26,2020-07-26,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",2020-08-20,2020-08-25,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",2020-08-20,2020-08-25,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [ ]:
df['inv_amt'] = df['inv_amt'].map(lambda x: float(x.replace(',', '')))

In [ ]:
df.sum()

vend_num       2002002002022022022022022022022032042042042252...
vend_name      United Parcel ServiceUnited Parcel ServiceUnit...
inv_amt                                                  22476.3
net_amt        75.0075.0075.00490.00239.00239.00239.00800.001...
description    Overnight document deliveryPostageOvernight do...
dtype: object

In [ ]:
df.to_csv('inv_lines.csv')